<a href="https://colab.research.google.com/github/apoorvapu/data_science/blob/main/NLP_languageTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# fine-tuning LLM for english to sanskrit translation

In [146]:
'''
from google.colab import drive
drive.mount('/content/drive')
# Read the API key from the file
with open("/content/drive/My Drive/hf_token.txt", "r") as f:
    hf_token = f.read().strip()

from huggingface_hub import login
login(hf_token)
'''

'\nfrom google.colab import drive\ndrive.mount(\'/content/drive\')\n# Read the API key from the file\nwith open("/content/drive/My Drive/hf_token.txt", "r") as f:\n    hf_token = f.read().strip()\n\nfrom huggingface_hub import login\nlogin(hf_token)\n'

In [147]:
'''
# Read the API key from the file
with open('/content/drive/MyDrive/wandb_key.txt', 'r') as f:
    wandb_key = f.read().strip()

# Set the W&B API key
import os
os.environ["WANDB_API_KEY"] = wandb_key
'''

'\n# Read the API key from the file\nwith open(\'/content/drive/MyDrive/wandb_key.txt\', \'r\') as f:\n    wandb_key = f.read().strip()\n\n# Set the W&B API key\nimport os\nos.environ["WANDB_API_KEY"] = wandb_key\n'

In [148]:
!pip install datasets bitsandbytes transformers

In [149]:
from peft import get_peft_model, LoraConfig, TaskType
import torch, gc
from datasets import load_dataset, Dataset

torch.cuda.empty_cache()
gc.collect()

533

In [150]:
# Load dataset
dataset = load_dataset("cfilt/iitb-english-hindi")  # Example: sanskrit-eng translation

README.md:   0%|          | 0.00/3.14k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [151]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [152]:
print(dataset['train'][0])

{'translation': {'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}


In [153]:
import pandas as pd

def convert_to_dataframe(dataset, data_type):
    # Extract the relevant data type (train, test, validation)
    data = dataset.get(data_type, [])

    # Check if data is available for the given type
    if not data:
        raise ValueError(f"No data found for {data_type}")

    # Extract the 'en' and 'sn' translations
    english_sentences = [entry['translation']['en'] for entry in data]
    sanskrit_sentences = [entry['translation']['hi'] for entry in data]

    # Create and return the DataFrame
    df = pd.DataFrame({
        'English': english_sentences,
        'Hindi': sanskrit_sentences
    })

    return df
train_df = convert_to_dataframe(dataset, 'train')
test_df = convert_to_dataframe(dataset, 'test')
validation_df = convert_to_dataframe(dataset, 'validation')
train_df.head()

,English,Hindi
0,Give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,Accerciser Accessibility Explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,The default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लग-इन खाका
3,The default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
4,A list of plugins that are disabled by default,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...


In [154]:
test_df.head()

,English,Hindi
0,A black box in your car?,आपकी कार में ब्लैक बॉक्स?
1,As America's road planners struggle to find th...,"जबकि अमेरिका के सड़क योजनाकार, ध्वस्त होते हुए..."
2,"The devices, which track every mile a motorist...","यह डिवाइस, जो मोटर-चालक द्वारा वाहन चलाए गए प्..."
3,The usually dull arena of highway planning has...,आम तौर पर हाईवे नियोजन जैसा उबाऊ काम भी अचानक ...
4,Libertarians have joined environmental groups ...,"आपने द्वारा ड्राइव किए गए मील, तथा संभवतः ड्रा..."


In [155]:
validation_df.head()

,English,Hindi
0,Students of the Dattatreya city Municipal corp...,महानगर पालिका अंतर्गत दत्तात्रय नगर माध्यमिक स...
1,With encouragement from Principal Sandhya Medp...,प्रधानाध्यापक संध्या मेडपल्लीवार के प्रोत्साहि...
2,"Rajesh Gavre, the President of the MNPA teache...",मनपा शिक्षक संघ के अध्यक्ष राजेश गवरे ने स्कूल...
3,Ramesh Saatpute examined the fort.,किले का परीक्षण रमेश सातपुते ने किया।
4,"Students like Nikhil Kavle, Darshan Gedekar, S...","किला निर्माण में निखिल कावले, दर्शन गेड़ेकर, स..."


In [156]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, BitsAndBytesConfig

# BitsAndBytes quantization config (4-bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"  # Better than 'fp4' for most tasks
)

# Load tokenizer and base model
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.


MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear4bit(in_features=512, out_features=384, bias=False)
              (k): Linear4bit(in_features=512, out_features=384, bias=False)
              (v): Linear4bit(in_features=512, out_features=384, bias=False)
              (o): Linear4bit(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear4bit(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear4bit(in_features=512, out_features=1024

In [157]:
# Define your input English sentence
english_text = "The king went to the forest"

# mT5 expects a prompt indicating the task. Let's try:
input_text = f"translate English to Hindi: {english_text}"

# Tokenize input
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate output
outputs = model.generate(inputs, max_length=40, num_beams=4, early_stopping=True)

# Decode output
translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"translation: {translation}")

translation: <extra_id_0>.


### so currently it cannot translate. Will fine-tune

In [158]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model.to(device)

trainable params: 172,032 || all params: 300,348,800 || trainable%: 0.0573


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MT5ForConditionalGeneration(
      (shared): Embedding(250112, 512)
      (encoder): MT5Stack(
        (embed_tokens): Embedding(250112, 512)
        (block): ModuleList(
          (0): MT5Block(
            (layer): ModuleList(
              (0): MT5LayerSelfAttention(
                (SelfAttention): MT5Attention(
                  (q): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=4, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=4, out_features=384, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()


In [159]:
train_df = Dataset.from_pandas(train_df)
validation_df = Dataset.from_pandas(validation_df)
test_df = Dataset.from_pandas(test_df)

def preprocess_function(example):
    # Tokenize the inputs and labels
    inputs = tokenizer(example['English'], padding="max_length", truncation=True, max_length=40, return_tensors="pt")
    labels = tokenizer(example['Hindi'], padding="max_length", truncation=True, max_length=40, return_tensors="pt")

    # Add the labels to the inputs
    inputs["labels"] = labels["input_ids"]
    return inputs

In [160]:
tokenized_train = train_df.map(preprocess_function, batched=True)
tokenized_val = validation_df.map(preprocess_function, batched=True)
tokenized_test = test_df.map(preprocess_function, batched=True)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [161]:
tokenized_test

Dataset({
    features: ['English', 'Hindi', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2507
})

In [162]:
tokenized_train

Dataset({
    features: ['English', 'Hindi', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1659083
})

In [163]:
print(tokenized_train[0])

{'English': 'Give your application an accessibility workout', 'Hindi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें', 'input_ids': [75259, 772, 13523, 461, 259, 262, 164216, 259, 90211, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [3192, 757, 9608, 35551, 1053, 4510, 8573, 14440, 110501, 1114, 58082, 175330, 1163, 9558, 6350, 5197, 2917, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [164]:
print(tokenized_train[1])

{'English': 'Accerciser Accessibility Explorer', 'Hindi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक', 'input_ids': [46502, 155198, 3406, 298, 164216, 32672, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [60182, 2255, 21954, 20412, 12268, 4510, 8573, 14440, 110501, 1114, 39883, 17950, 205617, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [165]:
torch.cuda.empty_cache()
gc.collect()

# Data Collator handles padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./mt5_sanskrit_lora",
    per_device_train_batch_size=40,
    per_device_eval_batch_size=40,
    evaluation_strategy="steps",
    eval_steps=500,  # evaluate every 100 steps
    logging_steps=500,
    save_steps=500,
    save_total_limit=1,
    learning_rate=2e-5,
    num_train_epochs=1,   # doesn't matter if you use max_steps For large datasets like this (75k samples), 1 epoch = ~9,400 steps if batch size = 8.
    max_steps=10000,        # 🚨 Training stops after 500 steps With max_steps=500, it will override num_train_epochs.
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,  # Ensures best model is saved at the end of training
    metric_for_best_model="eval_loss",  # Specify the metric to monitor for early stopping
    push_to_hub=False,
    report_to="none"
)

from transformers import EarlyStoppingCallback
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-165-65e676b91cb8>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [166]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.000000,60.231915


KeyboardInterrupt: 

In [ ]:
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to("cuda")
    output = model.generate(**inputs, max_length=40, num_beams=5, early_stopping=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)

print(translate("The king went to the forest"))


In [ ]:
# **Evaluate the Model**
results = trainer.evaluate(test_dataset)
print("Evaluation Results:", results)

In [ ]:

# Define the preprocessing function
def hi(examples):
    # Tokenize the English sentences (inputs) and return tensors
    inputs = tokenizer(examples['text'], padding="max_length", max_length=64, truncation=True, return_tensors="pt")
    return inputs

# Data: English sentences
data = ["I am very happy today and hungry too.", "Life is good.", "All is well."]

# Wrap it in a dataset
dataset = Dataset.from_dict({"text": data})

# Apply the preprocessing function
dataset = dataset.map(hi, batched=True)

# Get inputs from the dataset
inputs = dataset['input_ids']  # Extract the input ids after preprocessing

# Convert the list of input_ids into a tensor and move to the appropriate device
inputs_tensor = torch.tensor(inputs).to(device)  # Move to device (GPU or CPU)

# Make predictions
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs_tensor,
        max_length=128,
        num_beams=5,
        temperature=0.7,  # Adjust temperature for diversity
        top_p=0.9,  # nucleus sampling
        top_k=50,  # Limit the top-k tokens to sample from
        early_stopping=True
    )

# Decode the predictions (Sanskrit translations)
decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print the translations
for i, text in enumerate(decoded_preds):
    print(f"English: {data[i]}")
    print(f"Sanskrit Translation: {text}")
    print('-' * 50)
